In [1]:
import numpy as np
import plotly
import logging
logging.getLogger().setLevel(logging.INFO)

from idst_util import trivial
from idst_util import dstc2
from operator import itemgetter
from plotly.graph_objs import Bar, Layout
from plotly.graph_objs.layout import Margin
plotly.offline.init_notebook_mode(connected = True)

[nltk_data] Downloading package punkt to /home/is/andrei-
[nltk_data]     cc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
# Make sure data is available
trivial.print_idst()
dstc2.check()

# Retrieve raw data
raw_X_train, raw_Y_train, \
raw_X_dev, raw_Y_dev, \
raw_X_test, raw_Y_test, \
ontology = dstc2.retrieve_raw_datasets(only_transcript = False)

INFO:root:+--------------------------------+
INFO:root:|         _ ____  ___________    |
INFO:root:|        (_) __ \/ ___/_  __/    |
INFO:root:|       / / / / /\__ \ / /       |
INFO:root:|      / / /_/ /___/ // /        |
INFO:root:|     /_/_____//____//_/         |
INFO:root:|                                |
INFO:root:+--------------------------------+
INFO:root:|Incremental Dialog State Tracker|
INFO:root:+--------------------------------+
INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|         Data Checker           |
INFO:root:+--------------------------------+
INFO:root:Looking for dstc2 directory in .
INFO:root:dstc2 was found!
INFO:root:Looking for dstc2_traindev directory in ./dstc2
INFO:root:dstc2_traindev was found!
INFO:root:Looking for dstc2_test directory in ./dstc2
INFO:root:dstc2_test was found!
INFO:root:Looking for dstc2_scripts directory in ./dstc2
INFO:root:dstc2_scripts was found!
INFO:root:Done!
INFO:root:+-

INFO:root:Extracting raw dev features


INFO:root:Reading dstc2_test.flist
INFO:root:Asserted 1117 dialogs for dstc2_test.flist
INFO:root:Extracting raw test features


INFO:root:Done!


In [14]:
def retrieve_info(raw_X, only_user_utterance = False):
    
    token_to_count = {}
    max_sequence_length = 0
    turns_list = []
    
    for raw_dialog in raw_X:
        turns_count = 0
        for raw_turn in raw_dialog["turns"]:
            turns_count += 1
            if only_user_utterance:
                tokens_scores = raw_turn["user"]
            else:    
                tokens_scores = raw_turn["system"] + raw_turn["user"]
            for token_score in tokens_scores:
                token = token_score[0]
                if token not in token_to_count:
                    token_to_count[token] = 1
                else:
                    token_to_count[token] += 1
            turns_list.append(turns_count)
            turn_length = len(tokens_scores)
            if turn_length > max_sequence_length:
                max_sequence_length = turn_length
    
    return token_to_count, max_sequence_length, np.mean(turns_list)

In [15]:
train_vocabulary, \
train_max_sequence_length, \
train_average_turns_per_dialog = retrieve_info(raw_X_train)


dev_vocabulary, \
dev_max_sequence_length, \
dev_average_turns_per_dialog = retrieve_info(raw_X_dev)

test_vocabulary, \
test_max_sequence_length, \
test_average_turns_per_dialog = retrieve_info(raw_X_test)

In [17]:
logging.info("+--------------------------------+")
logging.info("|     Dialog State Tracker 2     |")
logging.info("|         Data Analysis          |")
logging.info("+--------------------------------+\n")

logging.info("Train number of dialogs:\t{}".format(len(raw_X_train)))
logging.info("Train vocabulary length:\t{}".format(len(train_vocabulary)))
logging.info("Train max sequence length:\t{}".format(train_max_sequence_length))
logging.info("Train average turns:\t\t{}\n".format(train_average_turns_per_dialog))

logging.info("Dev number of dialogs:\t{}".format(len(raw_X_dev)))
logging.info("Dev vocabulary length:\t{}".format(len(dev_vocabulary)))
logging.info("Dev max sequence length:\t{}".format(dev_max_sequence_length))
logging.info("Dev average turns:\t\t{}\n".format(dev_average_turns_per_dialog))

logging.info("Test number of dialogs:\t{}".format(len(raw_X_test)))
logging.info("Test vocabulary length:\t{}".format(len(test_vocabulary)))
logging.info("Test max sequence length:\t{}".format(test_max_sequence_length))
logging.info("Test average turns:\t\t{}\n".format(test_average_turns_per_dialog))

INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|         Data Analysis          |
INFO:root:+--------------------------------+

INFO:root:Train number of dialogs:	1612
INFO:root:Train vocabulary length:	896
INFO:root:Train max sequence length:	28
INFO:root:Train average turns:		4.930461591162113

INFO:root:Dev number of dialogs:	506
INFO:root:Dev vocabulary length:	720
INFO:root:Dev max sequence length:	25
INFO:root:Dev average turns:		5.454499237417387

INFO:root:Test number of dialogs:	1117
INFO:root:Test vocabulary length:	892
INFO:root:Test max sequence length:	27
INFO:root:Test average turns:		5.983923154701719



In [39]:
train_X_axis = []
train_Y_axis = []
train_top_tokens = []
sorted_dict = sorted(train_vocabulary.items(), key = itemgetter(1), reverse = True)

for token, token_count in sorted_dict:
    if len(train_top_tokens) <= 10:
        train_top_tokens.append((token, token_count))
    if token_count > 100:
        train_X_axis.append(token)
        train_Y_axis.append(token_count)

assert(len(train_X_axis) == len(train_Y_axis)) 

plotly.offline.iplot({
    "data": [Bar(orientation = "v",
                 x = train_X_axis,
                 y = train_Y_axis,
                 marker = dict(color = "#1abc9c"))],
    "layout": Layout(title = "<b>Train tokens distribution</b>",
                     xaxis = dict(title = "<b>Tokens</b>", dtick = 1),
                     yaxis = dict(title = "<b>Frequency</b>"),
                     margin = Margin(b = 180)
                    )
})

logging.info("Train top tokens:\t{}".format(train_top_tokens))

INFO:root:Train top tokens:	[('inform', 7936), ('name', 6162), ('offer', 6077), ('food', 5665), ('area', 3331), ('the', 2768), ('pricerange', 2695), ('phone', 2571), ('request', 1910), ('slot', 1910), ('i', 1904)]


In [38]:
dev_X_axis = []
dev_Y_axis = []
dev_top_tokens = []
sorted_dict = sorted(dev_vocabulary.items(), key = itemgetter(1), reverse = True)

for token, token_count in sorted_dict:
    if len(dev_top_tokens) <= 10:
        dev_top_tokens.append((token, token_count))
    if token_count > 40:
        dev_X_axis.append(token)
        dev_Y_axis.append(token_count)

assert(len(dev_X_axis) == len(dev_Y_axis)) 

plotly.offline.iplot({
    "data": [Bar(orientation = "v",
                 x = dev_X_axis,
                 y = dev_Y_axis,
                 marker = dict(color = "#3498db"))],
    "layout": Layout(title = "<b>Dev tokens distribution</b>",
                     xaxis = dict(title = "<b>Tokens</b>", dtick = 1),
                     yaxis = dict(title = "<b>Frequency</b>"),
                     margin = Margin(b = 180)
                    )
})


logging.info("Dev top tokens:\t{}".format(dev_top_tokens))

INFO:root:Dev top tokens:	[('inform', 2448), ('name', 2039), ('offer', 1997), ('food', 1929), ('area', 1124), ('the', 944), ('pricerange', 876), ('phone', 775), ('i', 701), ('request', 631), ('slot', 631)]


In [36]:
test_X_axis = []
test_Y_axis = []
test_top_tokens = []
sorted_dict = sorted(test_vocabulary.items(), key = itemgetter(1), reverse = True)

for token, token_count in sorted_dict:
    if len(test_top_tokens) <= 10:
        test_top_tokens.append((token, token_count))
    if token_count > 70:
        test_X_axis.append(token)
        test_Y_axis.append(token_count)

assert(len(test_X_axis) == len(test_Y_axis)) 

plotly.offline.iplot({
    "data": [Bar(orientation = "v",
                 x = test_X_axis,
                 y = test_Y_axis,
                 marker = dict(color = "#9b59b6"))],
    "layout": Layout(title = "<b>Test tokens distribution</b>",
                     xaxis = dict(title = "<b>Tokens</b>", dtick = 1),
                     yaxis = dict(title = "<b>Frequency</b>"),
                     margin = Margin(b = 180)
                    )
})


logging.info("Test top tokens:\t{}".format(test_top_tokens))

INFO:root:Test top tokens:	[('inform', 6188), ('food', 5831), ('name', 3978), ('offer', 3824), ('area', 3037), ('pricerange', 2393), ('request', 2174), ('slot', 2174), ('the', 2104), ('impl-conf', 1872), ('phone', 1765)]


In [40]:
train_dev_oov_tokens = []

for token, count in dev_vocabulary.items():
    if token not in train_vocabulary:
        if token not in train_dev_oov_tokens:
            train_dev_oov_tokens.append(token)

train_dev_oov_rate = len(train_dev_oov_tokens)/len(dev_vocabulary)

logging.info("Train-Dev OOV rate:\t{}\n".format(train_dev_oov_rate))
logging.info("Train-Dev OOV tokens:\t{}".format(train_dev_oov_tokens))

INFO:root:Train-Dev OOV rate:	0.1375

INFO:root:Train-Dev OOV tokens:	['allowed', 'quarter', 'venue', 'such', 'essentially', 'best', 'twenty', 'search', 'touch', 'move', 'typical', 'direct', 'comments', 'wo', 'noon', 'alone', 'americas', 'the americas', 'recognised', 'drink', 'clowns cafe', 'exchange', '01223 324351', 'wine', 'queen', 'nice', 'eclectic', 'third', 'found', 'worry', 'twelve', 'global', 'club', 'hold', 'drop', 'affordable', 'helpful', 'bridge', 'miles', 'available', '52 mill road city centre', 'ranges', 'bennys', 'connection', 'brb', 'goes', 'cow', "'re", 'two', 'point', 'mentioned', 'below', 'clowns', 'behind', 'cuisine', 'seen', 'answer', 'pitch', 'recommend', 'eleven', 'tight', 'children', "'ve", 'exact', '106 mill road city centre', 'theme', 'content', 'has', 'lasts', 'uno', 'cambridge city football club milton road chesterton', 'come', 'piece', 'true', 'seem', 'millers', 'avenue', 'luck', 'around', 'g', 'zealand', 'intermediate', 'cars', 'meze', 'tour', 'tells', 'arr

In [41]:
train_test_oov_tokens = []

for token, count in test_vocabulary.items():
    if token not in train_vocabulary:
        if token not in train_test_oov_tokens:
            train_test_oov_tokens.append(token)

train_test_oov_rate = len(train_test_oov_tokens)/len(test_vocabulary)

logging.info("Train-Test OOV rate:\t{}\n".format(train_test_oov_rate))
logging.info("Train-Test OOV tokens:\t{}".format(train_test_oov_tokens))

INFO:root:Train-Test OOV rate:	0.21973094170403587

INFO:root:Train-Test OOV tokens:	['count', '17', 'directly', '9', '3', 'tight', '4', 'wo', '109', 'fare', '31', '2', 'cow', '5', '15', '57', 'benny', 'spend', '12', 'includes', 'essentially', 'uno', 'quickly', 'two', 'excuse', '69', '22', 'make', 'suggestions', 'ho', 'eight', 'spent', 'spain', 'still', 'seen', 'up', 'county', "'re", 'p', 'such', 'meal', 'books', 'dear', 'wine', 'hong', 'third', '11', 'hotel', 'storey', 'wow', 'advice', 'confirm', 'salad', 'comment', 'remember', 'h', 'nice', 'stops', 'useless', 'found', 'connect', 'cheapest', 'longer', 'kinds', 'itself', 'reference', '7', 'tomorrow', 'continues', 'signature', 'huntingdon road city centre', 'thru', 'fill', 'kings', 'pool', 'union', 'twenty', '110', 'has', '56', 'movies', 'forty', 'helpful', 'hate', 'seem', 'names', 'pig', '10', 'quick', 'green', 'straight', 'homerton', 'checking', 'beers', 'collecting', 'continue', 'nothing', 'feature', 'kettle', 'granta', 'selling', 's